### Семинар 4. Генерация текста: рекуррентные нейронные сети

В этом семинаре мы продолжим учиться генерировать анекдоты. На этот раз с помощью LSTM. Датасет взят [отсюда](https://t.me/NeuralShit/2321).

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import re
import torch
from tqdm.auto import tqdm

### Датасет

In [3]:
with open('anek.txt', 'r') as f:
    aneki = f.read().strip().replace('<|startoftext|>', '').split('\n\n')

In [4]:
aneki[42]

'Кто сказал, что солдат мечтает стать генералом? Солдат мечтает стать хлеборезом.'

Поделим выборку на тренировочную и тестовую.

In [6]:
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split

In [7]:
train_texts, test_texts = train_test_split(aneki, test_size=0.1)

In [8]:
train_loader = DataLoader(train_texts, batch_size=128, shuffle=True, pin_memory=True)
test_loader = DataLoader(train_texts, batch_size=128, shuffle=False, pin_memory=True)

### Токениация

В качестве токенизатора возьмем предобученный из библиотеки [hugging face](https://huggingface.co). Он имеет относительно небольшой размер словаря, что хорошо для нашей легковестной модели.

In [9]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained("cointegrated/rubert-tiny")
len(tokenizer)

29564

### LSTM

LSTM – это рекуррентная нейронная сеть, обладающая модулем памяти, который позволяет есть улавливать долгосрочные зависимости. Ее скрытое состояние в вектор памяти обновляются на каждом шаге по следующим формулам:

$$
\begin{gathered}
f_t=\sigma\left(W_f \cdot\left[h_{t-1}, x_t\right]+b_f\right) \\
i_t=\sigma\left(W_i \cdot\left[h_{t-1}, x_t\right]+b_i\right) \\
o_t=\sigma\left(W_o \cdot\left[h_{t-1}, x_t\right]+b_o\right) \\
\tilde{C}_t=\tanh \left(W_c \cdot\left[h_{t-1}, x_t\right]+b_c\right) \\
C_t=f_t \odot C_{t-1}+i_t \odot \tilde{C}_t \\
h_t=o_t \odot \tanh \left(C_t\right)
\end{gathered}
$$

Мы не будем учить модель, потому что на это уйдет весь семинар, поэтому возьмем заранее обученную.

In [10]:
from model import LSTM
from train import train
from train import evaluate

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

lstm = LSTM(len(tokenizer), input_size=256, hidden_size=256).to(device)

In [12]:
lstm.load_state_dict(torch.load('aneki_lstm_256.pt'))

<All keys matched successfully>

In [13]:
from train import evaluate

test_accuracy = evaluate(lstm, test_loader, tokenizer)
test_accuracy

  0%|          | 0/873 [00:00<?, ?it/s]

0.3197522737589352

In [20]:
from utils import perplexity

ppl = perplexity(lstm, train_texts[:100], tokenizer)
ppl

/home/amshabalin/anaconda3/lib/python3.9/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='none' instead.
  warnings.warn(warning.format(ret))


tensor(32.7403, device='cuda:0')

## Оценка качества

В этой секции мы будем тестировать различные методы семплирования токенов. Для того, чтобы их можно было как-то сравнивать, необходимо ввести метрики качества. Мы будем отслеживать кросс-энтропийную ошибку большой и мощной языковой модели ([ruGPT](https://huggingface.co/ai-forever/rugpt3large_based_on_gpt2), если быть точнее), долю уникальных слов и энтропию предсказаний нашей модели.

Для начала посчитаем значения метрик для оригинального текста. Это нужно, чтобы мы знали, к каким цифрам стремиться. Оценивать качество будем по 5000 текстов.

In [14]:
n_texts = 5000

In [15]:
from utils import *

In [16]:
gpt_loss = GPTLoss(device)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [17]:
train_texts = []
for texts in train_loader:
    train_texts.extend(texts)

In [21]:
train_texts[:2]

['- Дима, давай быстрее собирайся, скоро начинаем открытие Олимпиады!- Вова? Я спать хочу!- Задрал, там поспишь!',
 'Работник мясокомбината пытался вынести со склада продукцию, и был пойман с потрохами.']

In [26]:
unique_words_rate(train_texts[:n_texts])

0.2522258276845618

In [28]:
source_gpt_loss = gpt_loss.compute(train_texts[:n_texts])
source_gpt_loss

  0%|          | 0/5000 [00:00<?, ?it/s]

3.5969808605536358

Реализуем функцию для генерации текста. Одно из приятных свойств рекуррентных нейронных сетей заключается в том, что для предсказания следующего токена нам не нужно пробегать всю последовательность заново. Мы можем просто подать модели на вход последний сгенерированный токен и текущее скрытое состояние.

In [18]:
@torch.no_grad()
def generate(model, sampler, n_samples=1):
    model.eval()

    cur_tokens = torch.full((n_samples, 1), tokenizer.cls_token_id, device=device)
    output_ids = cur_tokens.clone()

    h_t = torch.zeros(n_samples, 1, model.hidden_size, device=device)
    c_t = torch.zeros(n_samples, 1, model.hidden_size, device=device)

    all_entropies = []
    ended_texts = torch.zeros(n_samples)
    for i in range(256):
        output, (h_t, c_t) = model(cur_tokens, init_states=(h_t.squeeze(1), c_t.squeeze(1)))

        next_tokens, token_entropy = sampler.sample(output[:, -1].cpu())
        all_entropies.extend(token_entropy[ended_texts == 0])

        cur_tokens = next_tokens.unsqueeze(1).to(device)
        output_ids = torch.cat((output_ids, cur_tokens), dim=1)
        
        ended_texts += (next_tokens.cpu() == tokenizer.sep_token_id)
        if torch.all(ended_texts > 0):
            break

    return tokenizer.batch_decode(output_ids), np.mean(all_entropies)

def generate_n_texts(model, sampler, n_texts=1000):
    batch_size = 250
    generated_texts = []
    entropies = []
    for _ in tqdm(range(0, n_texts // batch_size)):
        texts, entropy = generate(model, sampler, n_samples=batch_size)
        generated_texts.extend(clear_text(texts))
        entropies.append(entropy)
        
    return generated_texts, np.mean(entropies)

В классе `Sampler` реализованны разничные способы семлирования токенов. Код этого класса посмотреть нельзя, так как задание на реализацию этих способов будет в домашке.

Начнем с жадного метода сэмплирования.

In [19]:
from sampler import Sampler

In [30]:
sampler = Sampler(prob_mode='temperature', sample_mode='greedy', temp=1)

generated_texts, entropy = generate_n_texts(lstm, sampler, n_texts=n_texts)

  0%|          | 0/20 [00:00<?, ?it/s]

In [31]:
generated_texts[:3]

['- А ты знаешь, что у вас есть? - Да, а что? - А ты что, не знаешь, что это такое? - Да, а что? - А я не могу...',
 '- А ты знаешь, что у вас есть? - Да, а что? - А ты что, не знаешь, что это такое? - Да, а что? - А я не могу...',
 '- А ты знаешь, что у вас есть? - Да, а что? - А ты что, не знаешь, что это такое? - Да, а что? - А я не могу...']

Так как никакого элемента случайности в генерации нет, тексты получились одинаковыми.

In [71]:
unique_words_rate(generated_texts), entropy

(0.000104, 0.0)

In [38]:
lstm_gpt_loss = gpt_loss.compute(generated_texts)
lstm_gpt_loss

  0%|          | 0/5000 [00:00<?, ?it/s]

2.181544780731201

Конечно же, много уникальных токенов в 5000 одинаковых текстах не найдется. Энтропию тоже нет смысла считать. Ошибка большой языковой модели оказалась ниже, чем у оригинальных текстов, потому что сгенерированные тексты содержат повторения.

In [21]:
repetitions_gpt_loss = gpt_loss.compute(['Я не буду тратить мел. ' * 20])
repetitions_gpt_loss

  0%|          | 0/1 [00:00<?, ?it/s]

0.41997745633125305

#### Семплирование с температурой

Попробуем теперь семплировать токены случайно в соответствии с их вероятностями.

In [72]:
sampler = Sampler(prob_mode='temperature', sample_mode='random', temp=1)

generated_texts, entropy = generate_n_texts(lstm, sampler, n_texts=n_texts)

  0%|          | 0/20 [00:00<?, ?it/s]

In [73]:
generated_texts[:3]

['- Скажите, а почему пригласили вечеринвы за важной перед страхов Гаинец, а во мне встретили начинается как в будущем? - Возьмите жену в носа.',
 '- Девушка, вы откуда привыкрасным? - А куда вам закусить соседиб от тебя! - Я услышал, как в аду будут, чтобы на язык подожди радостноцизине советует, что вы кошек не знает чем с простого приснымся.',
 'Утро женщина - это когда с собой не появляется.']

In [74]:
unique_words_rate(generated_texts), entropy

(0.281033790117395, 3.6328158)

In [75]:
lstm_gpt_loss = gpt_loss.compute(generated_texts)
lstm_gpt_loss

  0%|          | 0/5000 [00:00<?, ?it/s]

6.998079306005547

Если взять температуру поменьше, то распределение вероятностей станет более вырожденным и, как следствие, энтропия уменьшится. При этом уменьшится и доля уникальных слов.

In [76]:
sampler = Sampler(prob_mode='temperature', sample_mode='random', temp=0.7)

generated_texts, entropy = generate_n_texts(lstm, sampler, n_texts=n_texts)

  0%|          | 0/20 [00:00<?, ?it/s]

In [77]:
generated_texts[:3]

['- Дура, ты чего такой длиннее, чем вас вернуть? - А в чем разница у тебя? - Я не знаю, что я тебе внутреннюю аптекарный.',
 'Бесит, когда в подъезде нельзя надо покупать.',
 '- Какие красивое, я - ни разу так красивыми? - А что это? - А что ты был? - Да, я ношу и открывает бухгалтера.']

In [78]:
unique_words_rate(generated_texts), entropy

(0.13245934610384832, 2.3059564)

In [79]:
lstm_gpt_loss = gpt_loss.compute(generated_texts)
lstm_gpt_loss

  0%|          | 0/5000 [00:00<?, ?it/s]

5.407580097166828

Соответственно, если увеличить температуру, распределение вероятностей станет более равномерным и энтропия увеличится. Так же, редкие слова начнут семплироваться чаще.

In [80]:
sampler = Sampler(prob_mode='temperature', sample_mode='random', temp=1.5)

generated_texts, entropy = generate_n_texts(lstm, sampler, n_texts=n_texts)

  0%|          | 0/20 [00:00<?, ?it/s]

In [81]:
generated_texts[:3]

['Новый год начали выпускают женщин о Трой часы для к ней в маршрутке, вылавиши себе новую сумку цески берет документы этой, дня назад два версии Калипрасно вышкосоксисляют чем Николай.',
 'Сеев змея как Украине звый бралася перед Потапдому музыку починит к любой бум гаечности по Зеленоре, Мэнбек нет я быстрокотсия с тобой говорить им один самогоусенников.',
 'Це несла такие та! Это уже перек мастериажат соглашение стройные тяжеляли давно в карманах Майкловые черном...., этим хотел правители. Hе женщины им член Ивановна.ил. Остантирушка игр в предкле искусств можно.']

In [82]:
unique_words_rate(generated_texts), entropy

(0.35940269370071326, 5.8276668)

In [83]:
lstm_gpt_loss = gpt_loss.compute(generated_texts)
lstm_gpt_loss

  0%|          | 0/5000 [00:00<?, ?it/s]

8.564157451305716

#### topk семплирование

Тут применяется схожая логика со случаем семплирования с температурой. Выбирая небольшое k, мы начинаем семплировать только самые вероятные токены, и такое поведение отражается на метриках.

In [84]:
sampler = Sampler(prob_mode='topk', sample_mode='random', k=50)

generated_texts, entropy = generate_n_texts(lstm, sampler, n_texts=n_texts)

  0%|          | 0/20 [00:00<?, ?it/s]

In [85]:
generated_texts[:3]

['- Скажите, вы уже сзади! - Это чео! Я не хочу посоветовать.',
 'Девушка, вы так себе думаешь, что в этом году в России стали меньше вечной сексу с медведя. Может, это не знал, что вы поняли, купи мне колгот?',
 '- Вовочка, ты чего собираешься? - Да, внучек! Я считаю!']

In [86]:
unique_words_rate(generated_texts), entropy

(0.18469160722068031, 2.6243415)

In [87]:
lstm_gpt_loss = gpt_loss.compute(generated_texts)
lstm_gpt_loss

  0%|          | 0/5000 [00:00<?, ?it/s]

6.250501114559838

Увеличивая k, мы понижаем вероятность каждого отдельного токена, и это тоже видно на метриках.

In [88]:
sampler = Sampler(prob_mode='topk', sample_mode='random', k=500)

generated_texts, entropy = generate_n_texts(lstm, sampler, n_texts=n_texts)

  0%|          | 0/20 [00:00<?, ?it/s]

In [89]:
generated_texts[:3]

['В Одессе засматривая Чака молодецмана России, это прекращается по ночам : - Может ли вы зеркало - то, что смогу жить сейчас на ужином? - Нет, я а это лучше, чем вы ума, выбор еще и дают.',
 'Запилишь работу так, как будто выучил опытный компьютерщика или поменько, а потом вызвали у девяного испанию.',
 'В магазине : - Сидоров, ты откуда и так получилось за нас замечали, то мне было 30 лет? - Плохой старуг в первый раз - стоишь трубку.']

In [90]:
unique_words_rate(generated_texts), entropy

(0.26616300060172493, 3.45268)

In [91]:
lstm_gpt_loss = gpt_loss.compute(generated_texts)
lstm_gpt_loss

  0%|          | 0/5000 [00:00<?, ?it/s]

6.9103927530825775

#### Nucleus sampling

Значение 0.95 для параметра p считается самым удачным, для нашей модели оно тоже отлично подходит с точки зрения доли уникальных слов.

In [ ]:
sampler = Sampler(prob_mode='temperature', sample_mode='nucleus', temp=1, p=0.95)

generated_texts, entropy = generate_n_texts(lstm, sampler, n_texts=1000)

In [126]:
generated_texts[:3]

['Многие ученые появится парочка жениху, когда он пропилишься без трусики куплю - с собой лотереться, давай влюбленная плавание нации.',
 'Мой друг спрашивают : - Что твоя мама в парад - финансы, которая водка? - Только когда я собака... Семнадцать вольманная взгляда, не интересуется, ты ему ответила в целом замельтеку...',
 'Шашен национальности, вы не делай олимпийским миллионеры : им предупреди счастливой рабочие дни в эфир.']

In [127]:
unique_words_rate(train_texts[:1000]), unique_words_rate(generated_texts), entropy

(0.39821234171775627, 0.38607075913776945, 3.2875628)

In [128]:
lstm_gpt_loss = gpt_loss.compute(generated_texts)
lstm_gpt_loss

  0%|          | 0/1000 [00:00<?, ?it/s]

6.92559085117137

Уменьшение p приведет нас к менее разнообразному тексту.

In [ ]:
sampler = Sampler(prob_mode='temperature', sample_mode='nucleus', temp=1, p=0.7)

generated_texts, entropy = generate_n_texts(lstm, sampler, n_texts=1000)

In [112]:
generated_texts[:3]

['Колдунья в примету : - У тебя жена звонит другу, так не кусает! - Молодой человек, вы круточку на 8 марта?',
 '- Ты куда тебя поймал? - Ты понимаешь, какой ты мне подумать?! - Да, возьмите...',
 'Объявление. Вова, когда он приходит домой, отбирается с родителями и собирали им дочке : - И как же сейчас придется?! - Да...']

In [113]:
unique_words_rate(generated_texts), entropy

(0.2524177949709865, 2.2645855)

In [114]:
lstm_gpt_loss = gpt_loss.compute(generated_texts)
lstm_gpt_loss

  0%|          | 0/1000 [00:00<?, ?it/s]

5.8342101303367